# OpenAI API Demo

In [ ]:
# @title Install OpenAI & PYPDF2 Libraries

%pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

%pip install PyPDF2

# Retrival Augmented Generation Application (RAG) Demo (By Dhruv)
**What We Will Do:**  We will create a RAG Application that can be used as a math tutor to help walk you through solutions to the UKMT Intermediate Maths Challenge

**The Data That We Will Use:**

https://ukmt.org.uk/wp-content/uploads/2024/02/IMC_2024-Paper.pdf

⬆ A PDF of Maths Competition Past Papers ⬆

**What is Retrival Augmented Generation?**

Standalone LLMs (including ChatGPT) often encounter limitations when tasked with specific queries. These limitations stem from their inability to access external information or context beyond their pre-trained knowledge.

To overcome this challenge, researchers have developed innovative approaches like Retrieval Augmented Generation (RAG). Motivated by the need to enhance language models with access to external knowledge sources, RAG combines the power of traditional language models with retrieval mechanisms, enabling them to generate more informed and contextually relevant responses.

In [ ]:
# @title Install OpenAI & PYPDF2 Libraries

%pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

%pip install PyPDF2

In [ ]:
# @title  Download Our Data

# you can put in a link to another PDF as well, but remember to also change the pdf_name variable when you do that and the system prompt

!wget https://ukmt.org.uk/wp-content/uploads/2024/02/IMC_2024-Paper.pdf

pdf_name = 'IMC_2024-Paper.pdf'

In [ ]:
# @title Step 1: Import Libraries

from PyPDF2 import PdfReader

from openai import OpenAI

In [ ]:
# @title Step 2: Read PDF File

pdf = PdfReader(pdf_name)


text = ""


for page_num in range(len(pdf.pages)):

    page = pdf.pages[page_num]

    text += page.extract_text()

    text += ' '

In [ ]:
# @title Step 3: Create ask_question Function


messages = []


def ask_question(question):


    print('\n')


# you can replace this OpenAI API key with your own

    api_key = ''


    client = OpenAI(api_key=api_key)


    response = client.chat.completions.create(model = 'chatgpt-4o-latest', messages = messages, temperature=0.1, stream=True)


# streaming (typing) effect in the output

    out = ''

    div = 0

    for chunk in response:

          if chunk.choices[0].delta.content is not None:

              content_delta = chunk.choices[0].delta.content

              content_delta = content_delta.replace('*', '')

              content_delta = content_delta.replace('#', '')

              content_delta = content_delta.replace("\\", '')

              content_delta = content_delta.replace("frac", '')

              content_delta = content_delta.replace("{", '')

              if '}' in content_delta and div == 0:
                content_delta = content_delta.replace("}", '/')
                div += 1
              else:
                content_delta = content_delta.replace("}", '')
                div -= 0

              print(content_delta, end="")

              out += content_delta

              if len(content_delta) - 1   == ('.' or ',' or '!'):

                print('\n')

    print('\n')


    return out

In [ ]:
# @title Step 4: Run Program

while True:


      question = input("\n  User:  ")


      if question.lower() == 'exit':

          messages.clear()

          break


      messages.append({'role':'system',
             'content':f'you are helping students understand {question} problems in this past paper: {text}, As an AI assistant, your role is to help students understand and solve math problems from their past papers. Instead of providing the answer outright, engage the student in a conversation for each step.'})


      answer = ask_question(question)


      messages.append({'role':'assistant', 'content':answer})

**Conclusion**

We have just implemented a very basic Retrieval Augmented Generation (RAG) system by providing the PDF text and the user's question as a single prompt to the language model. This approach works well for situations where the relevant context can fit within the model's context window. However, more complex RAG architectures are required when dealing with larger knowledge bases or multiple documents that cannot be concatenated and fit within the context window.

These advanced RAG systems typically have a separate retrieval component that efficiently searches through the knowledge base and retrieves the most relevant passages or documents based on the user's query. The retrieved context is then provided to the language model along with the query, allowing the model to generate a response based on the relevant information. Perplexity, for example, utilizes its own retrieval algorithm to find the most relevant context from its knowledge base, enabling it to provide accurate and contextually relevant responses even for complex queries spanning multiple documents or domains.